In [14]:
def build_listings_url(communes : list, 
                       cities : list, 
                       rooms : int, 
                       available_from : int,
                       maximum_price : int):
    base_url = "https://www.findbolig.nu/da-dk/find?"
    
    commune_string = f"{','.join(commune.replace(' ', '+') for commune in communes)}"
    cities_string = f"{','.join(city.replace(' ', '+') for city in cities)}"
    base_url = f"https://www.findbolig.nu/da-dk/find?Commune={commune_string}&Rooms={rooms}&City={cities_string}&AvailableFrom={available_from}&Rent={maximum_price}"

    return base_url
build_listings_url(["Frederiksberg"],
                   ["København Ø", "København V", "København K", "København N"],
                   2,
                   1,
                   11000)

'https://www.findbolig.nu/da-dk/find?Commune=Frederiksberg&Rooms=2&City=København+Ø,København+V,København+K,København+N&AvailableFrom=1&Rent=11000'

In [7]:
import json
import time

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from utils import scrape_and_soupify


def init_browser_and_scrape(url):
    firefox_options = Options()
    firefox_options.add_argument("--headless")  # Run Firefox in headless mode

    driver = webdriver.Firefox(options=firefox_options)
    driver.get(url)

    # driver.implicitly_wait(10)  # Wait for up to 10 seconds for the page to load
    wait = WebDriverWait(driver, 10)

    # Get the page source after the dynamic content has loaded
    html_content = driver.page_source

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")
    return soup

soup = init_browser_and_scrape("https://www.findbolig.nu/da-dk/find?Commune=K%C3%B8benhavn")

property_types = soup.find_all('div', {'class': 'property-card__spec', 'data-v-0133c15e': True})

In [6]:
email = "frederikpwr@gmail.com"
password = "hThfjwd$PZw^78"

def get_property_id(apt_url):
    soup = scrape_and_soupify(apt_url)
    property_id = soup.find('div', class_='c-property').get('id')
    return property_id

def request_apt_viewing(apt_url, email, password):
    front_url = "https://www.findbolig.nu"
    login_url = "https://www.findbolig.nu/api/authentication/login"
    cookie_url = "https://consent.app.cookieinformation.com/api/consent"

    property_id = get_property_id(apt_url)
    print(property_id)
    request_url = f"https://www.findbolig.nu/api/data/applications/residences/{property_id}"
    print(request_url)
    with requests.Session() as s:
        r = s.get(front_url) # load front page

        r = s.post(cookie_url,
                    data={"website_uuid":"1ac987a3-74b6-49b5-8c59-991f552c1355",
                        "timestamp":"2024-04-25T09:46:01.048Z",
                        "consent_url":"https://www.findbolig.nu/",
                        "consent_website":"fbn5prod.deas.dk",
                        "consent_domain":"www.findbolig.nu",
                        "user_uid":"b9f79c2e-9562-42f6-ae2e-cde16f0acd61",
                        "consents_approved":["cookie_cat_necessary"],
                        "consents_denied":["cookie_cat_functional","cookie_cat_statistic","cookie_cat_marketing","cookie_cat_unclassified"],
                        "user_agent":"Mozilla/5.0 (X11; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0"})

        r = s.post(login_url,
                    data={"email": email,
                        "password": password,})

        r = s.post(request_url)
        print(r.text)
        # r = s.delete(request_url) # delete application
        
    if r.status_code == 200:
        if r.text == "Cancel":
            print("Succesfully requested an apartment viewing .")
        elif r.text == "Apply":
            print("Succesfully cancelled apartment viewing request.")
        elif r.text == "Disabled":
            print("Apartment viewing are not enabled for the giving apartment.")
        else:
            print("Unknown response.")
    else:
        raise ConnectionError(f"Could not connect to Findbolig to {apt_url}.")

request_apt_viewing("https://www.findbolig.nu/da-dk/residence/92082", email, password)



095bc107-7480-4a33-962d-fbd115d486b1
https://www.findbolig.nu/api/data/applications/residences/095bc107-7480-4a33-962d-fbd115d486b1
"Disabled"
Message send succesfully!


In [6]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType

driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))

# driver = webdriver.Chrome()
driver.get("https://www.google.com/")
driver.quit()

/bin/sh: line 1: chromium: command not found
/bin/sh: line 1: chromium-browser: command not found
/bin/sh: line 1: chromium: command not found
/bin/sh: line 1: chromium-browser: command not found


AttributeError: 'NoneType' object has no attribute 'split'

In [58]:
import requests
import json

def get_listings(COMMUNES : list, CITIES : list, MIN_ROOMS : int, AVAILABLE_FROM : str, MAXIMUM_PRICE : int):
        data = {"pageSize":16,
        "page":0,
        "filters":{"Commune":COMMUNES,"City":CITIES,"Rent":f"[;{MAXIMUM_PRICE}]","Rooms":f"[{MIN_ROOMS};]","AvailableFrom":f"[;{AVAILABLE_FROM}T00:00:00Z]","Type":"Residence","TypeIndividualResidence":"true"},
        "mixedResults":False,
        "orderBy":"Created",
        "orderDirection":"DESC"}

        search_url = "https://www.findbolig.nu/api/search"

        with requests.Session() as s:
                r = s.post(search_url, json=data)

        if r.status_code == 200 and json.loads(r.text)["totalResults"] > 0:
                listings = json.loads(r.text)["results"]
                return listings

        else: 
                return None


COMMUNES = ["Frederiksberg"]
CITIES = ["København Ø", "København V", "København SV", "København K", "København N"]
MIN_ROOMS = 2
AVAILABLE_FROM = "2099-01-01"
MAXIMUM_PRICE = 12000

listings = get_listings(COMMUNES,
CITIES,
MIN_ROOMS,
AVAILABLE_FROM,
MAXIMUM_PRICE)
# print((listings))

for listing in listings:
        for key,  value in listing.items():
                # print(f"{key}: {value}")
                pass
        print(listing["uri"])

/residence/99447
/residence/98767


In [32]:
apt_data = {"uri" : listing["uri"],
            "location" : listing["freeStatuses"][0]["residenceAdvert"]["title"],
            "id" : listing["id"],
            "rent" : listing["rent"],
            "aconto" : listing["aconto"],
            "rooms" : listing["rooms"],
            "area" : listing["area"]
            }

False